In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [15]:
#ee.Authenticate()
#geemap.update_package()

Enter verification code: 4/1AY0e-g6yZ1pT0HkOg1nazV4AMY5B8uvsIQd2Q-e2TPfOe3YirdKtAXTelrs

Successfully saved authorization token.


In [ ]:
#converting javascript code to python - also checking to see if same issue occurs with map (weird white coloration):
#js_snippet = """
#var image1 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816'); 
#Map.addLayer(image1, {bands:['B4', 'B3', 'B2']}, 'ColorIR composite');
#"""
#geemap.js_snippet_to_py(js_snippet, add_new_cell = True, import_ee = True, import_geemap = True, show_map = True)

In [76]:
#list of plots in Virginia LTER dataset (not including broad creek)
plot_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-75.372740, 37.903114]), {'plot_id': 1}), #Assateague
  ee.Feature(ee.Geometry.Point([-75.671577, 37.625273]), {'plot_id': 2}), #Bellvue
  ee.Feature(ee.Geometry.Point([-75.876094, 37.395746]), {'plot_id': 3}), #Box_Tree
  ee.Feature(ee.Geometry.Point([-75.652248, 37.709249]), {'plot_id': 4}), #Channel_Point
  ee.Feature(ee.Geometry.Point([-75.942331, 37.175055]), {'plot_id': 5}), #Cushmans
  ee.Feature(ee.Geometry.Point([-75.940586, 37.167409]), {'plot_id': 6}), #Gator
  ee.Feature(ee.Geometry.Point([-75.814954, 37.485033]), {'plot_id': 7}), #Greens_Creek
  ee.Feature(ee.Geometry.Point([-75.672871, 37.451132]), {'plot_id': 8}), #Hog
  ee.Feature(ee.Geometry.Point([-75.900996, 37.346190]), {'plot_id': 9}), #Indiantown
  ee.Feature(ee.Geometry.Point([-75.558264, 37.784154]), {'plot_id': 10}), #Kegotank
  ee.Feature(ee.Geometry.Point([-75.861733, 37.416783]), {'plot_id': 11}), #Mill_Creek
  ee.Feature(ee.Geometry.Point([-75.822928, 37.468043]), {'plot_id': 12}), #N_Brownsville
  ee.Feature(ee.Geometry.Point([-75.929133, 37.287549]), {'plot_id': 13}), #Oyster
  ee.Feature(ee.Geometry.Point([-75.940690, 37.181277]), {'plot_id': 14}), #Steelmans
  ee.Feature(ee.Geometry.Point([-75.447000, 37.880000]), {'plot_id': 15}), #Wallops
  ee.Feature(ee.Geometry.Point([-75.819360, 37.481998]), {'plot_id': 16}) #Woodland_Farm
])


In [56]:
#landsat 5 surface reflectance images

roi = ee.Geometry.Point(-75.876094, 37.395746)

filteredCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS5.size().getInfo())
image0 = filteredCollectionLS5.first()
#print(image0.getInfo())

37


In [33]:
#determining cloud cover
cloudiness = image06.get('CLOUD_COVER')
print('CLOUD_COVER: ', cloudiness.getInfo())

CLOUD_COVER:  24


In [57]:
#get list of image ids in filtered collection
filteredCollectionLS5.aggregate_array('system:id').getInfo()

#figure out image dates!!

['LANDSAT/LT05/C01/T1_SR/LT05_014034_20060801',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20070804',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20070905',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20110831',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20030926',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20060918',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040912',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040827',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20090825',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20020907',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_201

In [61]:
#add image number for each of the images in the filtered collection - closer to 8/15 is best
image00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901') #pushed to later date bc clouds
image01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803') #clouds over some points
image02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20020822') #CLOUDS - make composite??? how
image03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825') #clouds?
image04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811')
image05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814')
image06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20060817') #image0 is 2006 - lower cloud coverage, maybe use?
image07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20070804')
image08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822')
image09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809')
image10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828') #CLOUDS?
image11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815')
#image12 = ee.Image()
#image13 = ee.Image()
#landsat 8?

In [79]:
#Mapping
#Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(image07, vis_param, "Landsat5SR")

Map.centerObject(roi, 10)
Map.set_plot_options(add_marker_cluster = True) #not sure what this does

#calculate ndvi
ndvi = image00.normalizedDifference(['B4', 'B3']).rename('NDVI')
vis_ndvi = {'min': -1, 
             'max': 1, 
             'palette': ['blue', 'white', 'green']}
#Map.addLayer(ndvi, vis_ndvi, "NDVI")

#add feature collection points for each plot
Map.addLayer(plot_pts)

Map

Map(center=[37.395746, -75.876094], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggle…

In [11]:
#extracting raster values for points; 
#from: https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points
#buffering points


In [75]:
#Exporting pixel values - any way to do this without manually selecting 
in_fc = plot_pts

#export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ndvipoints0.csv')
#out_shp = os.path.join(out_dir, 'points.shp') #shapefile exporting
geemap.extract_values_to_points(in_fc, image0, out_csv)

Generating URL ...
Please wait ...
Data downloaded to /Users/asariversjulien/Downloads/ndvipoints11.csv


In [39]:
print("hi")

In [13]:
print("hi")


hi
